In [0]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0 
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# **Retail Product Stockouts Prediction using AutoML Tables**

<table align="left">
  <td>
    <a href="https://cloud.google.com/ml-engine/docs/tensorflow/getting-started-keras">
      <img src="https://cloud.google.com/_static/images/cloud/icons/favicons/onecloud/super_cloud.png"
           alt="Google Cloud logo" width="32px"> Read on cloud.google.com
    </a>
  </td>
  <td>
    <a href="#">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="#">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## **Overview**

AutoML Tables enables you to build machine learning models based on tables of your own data and host them on Google Cloud for scalability. This Notebook demonstrates how you can use AutoML Tables to solve a product stockouts problem in the retail industry. This problem is solved using a binary classification approach, which predicts whether a particular product at a certain store will be out-of-stock or not in the next four weeks. Once the solution is built, you can plug this in with your production system and proactively predict stock-outs for your business.


### **Dataset**

In this solution, you will use two datasets: Training/Evaluation data and Batch Prediction inputs. To access the datasets in BigQuery, you need the following information.

##### **Training/Evaluation dataset**

  *  `Project ID: product-stockout`
  *  `Dataset ID: product_stockout`
  *  `Table ID: stockout`
  
##### **Batch Prediction inputs**

  *  `Project ID: product-stockout`
  *  `Dataset ID: product_stockout`
  *  `Table ID: batch_prediction_inputs`

##### **Data Schema**

<table align="left">
  <thead>
    <tr>
      <th> Field name </th>
      <th> Datatype </th>
      <th> Type </th>
      <th> Description </th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Item_Number</td>
      <td>STRING</td>
      <td>Identifier</td>
      <td>This is the product/ item identifier</td>
    </tr>
    <tr>
	<td>Category</td>
	<td>STRING</td>
	<td>Identifier</td>
	<td>Several items could belong to one category</td>
</tr>
<tr>
	<td>Vendor_Number</td>
	<td>STRING</td>
	<td>Identifier</td>
	<td>Product vendor identifier</td>
</tr>
<tr>
	<td>Store_Number</td>
	<td>STRING</td>
	<td>Identifier</td>
	<td>Store identifier</td>
</tr>
<tr>
	<td>Item_Description</td>
	<td>STRING</td>
	<td>Text Features</td>
	<td>Item Description</td>
</tr>
<tr>
	<td>Category_Name</td>
	<td>STRING</td>
	<td>Text Features</td>
	<td>Category Name</td>
</tr>
<tr>
	<td>Vendor_Name</td>
	<td>STRING</td>
	<td>Text Features</td>
	<td>Vendor Name</td>
</tr>
<tr>
	<td>Store_Name</td>
	<td>STRING</td>
	<td>Text Features</td>
	<td>Store Name</td>
</tr>
<tr>
	<td>Address</td>
	<td>STRING</td>
	<td>Text Features</td>
	<td>Address</td>
</tr>
<tr>
	<td>City</td>
	<td>STRING</td>
	<td>Categorical Features</td>
	<td>City</td>
</tr>
<tr>
	<td>Zip_Code</td>
	<td>STRING</td>
	<td>Categorical Features</td>
	<td>Zip-code</td>
</tr>
<tr>
	<td>Store_Location</td>
	<td>STRING</td>
	<td>Categorical Features</td>
	<td>Store Location</td>
</tr>
<tr>
	<td>County_Number</td>
	<td>STRING</td>
	<td>Categorical Features</td>
	<td>County Number</td>
</tr>
<tr>
	<td>County</td>
	<td>STRING</td>
	<td>Categorical Features</td>
	<td>County Name</td>
</tr>
<tr>
	<td>Weekly Sales Quantity</td>
	<td>INTEGER</td>
	<td>Time series data</td>
	<td>52 columns for weekly sales quantity from week 1 to week 52</td>
</tr>
<tr>
	<td>Weekly Sales Dollars</td>
	<td>INTEGER</td>
	<td>Time series data</td>
	<td>52 columns for weekly sales dollars from week 1 to week 52</td>
</tr>
<tr>
	<td>Inventory</td>
	<td>FLOAT</td>
	<td>Numeric Feature</td>
	<td>This inventory is stocked by the retailer looking at past sales and seasonality of the product to meet demand for future sales.</td>
</tr>
<tr>
	<td>Stockout</td>
	<td>INTEGER</td>
	<td>Label</td>
	<td>(1 - Stock-out, 0 - No stock-out) When the demand for four weeks future sales is not met by the inventory in stock we say we see a stock-out.
    <br/>This is because an early warning sign would help the retailer re-stock inventory with a lead time for the stock to be replenished.</td>
</tr>
  </tbody>
</table><br>
To use AutoML Tables with BigQuery you do not need to download this dataset. However, if you would like to use AutoML Tables with GCS you may want to download this dataset and upload it into your GCP Project storage bucket. 

**Instructions to download dataset:**

1. Sample Dataset: Download this dataset which contains sales data.

	* [Link to training data](https://console.cloud.google.com/bigquery?folder=&organizationId=&project=product-stockout&p=product-stockout&d=product_stockout&t=stockout&page=table):  

		Dataset URI: <bq://product-stockout.product_stockout.stockout>
	* [Link to data for batch predictions](https://console.cloud.google.com/bigquery?folder=&organizationId=&project=product-stockout&p=product-stockout&d=product_stockout&t=batch_prediction_inputs&page=table):  

		Dataset URI: <bq://product-stockout.product_stockout.batch_prediction_inputs>

2. Upload this dataset to GCS or BigQuery (optional). 

	* You could select either [GCS](https://cloud.google.com/storage/) or [BigQuery](https://cloud.google.com/bigquery/) as the location of your choice to store the data for this challenge. 

		1. Storing data on GCS: [Creating storage buckets, Uploading data to storage buckets](https://cloud.google.com/storage/docs/creating-buckets)
		2. Storing data on BigQuery: [Create and load data to BigQuery](https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui) (optional)


### **Objective**

#### **Problem statement**
A stockout, or out-of-stock (OOS) event is an event that causes inventory to be exhausted. While out-of-stocks can occur along the entire supply chain, the most visible kind are retail out-of-stocks in the fast-moving consumer goods industry (e.g., sweets, diapers, fruits). Stockouts are the opposite of overstocks, where too much inventory is retained.

#### **Impact**
According to a study by researchers Thomas Gruen and Daniel Corsten, the global average level of out-of-stocks within retail fast-moving consumer goods sector across developed economies was 8.3% in 2002. This means that shoppers would have a 42% chance of fulfilling a ten-item shopping list without encountering a stockout. Despite the initiatives designed to improve the collaboration of retailers and their suppliers, such as Efficient Consumer Response (ECR), and despite the increasing use of new technologies such as radio-frequency identification (RFID) and point-of-sale data analytics, this situation has improved little over the past decades.

The biggest impacts being

*   Customer dissatisfaction
*   Loss of revenue



#### **Machine Learning Solution**
Using machine learning to solve for stock-outs can help with store operations and thus prevent out-of-stock proactively.

There are three big challenges any retailer would face as they try and solve this problem with machine learning:

1.   Data silos: Sales data, supply-chain data, inventory data, etc. may all be in silos. Such disjoint datasets could be a challenge to work with as a machine learning model tries to derive insights from all these data points.
2.   Missing Features: Features such as vendor location, weather conditions, etc. could add a lot of value to a machine learning algorithm to learn from. But such features are not always available and when building machine learning solutions we think for collecting features as an iterative approach to improving the machine learning model.
3.  Imbalanced dataset: Datasets for classification problems such as retail stock-out are traditionally very imbalanced with fewer cases for stock-out. Designing machine learning solutions by hand for such problems would be time consuming effort when your team should be focusing on collecting features.

Hence, we recommend using AutoML Tables. With AutoML Tables you only need to work on acquiring all data and features, and AutoML Tables would do the rest. This is a one-click deploy to solving the problem of stock-out with machine learning.

### **Costs**

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage
* BigQuery
* AutoML Tables

Learn about [Cloud AI Platform pricing](https://cloud.google.com/ml-engine/docs/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing), [AutoML Tables pricing](https://cloud.google.com/automl-tables/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## **Set up your local development environment**

**If you are using Colab or AI Platform Notebooks**, your environment already meets
all the requirements to run this notebook. If you are using **AI Platform Notebook**, make sure the machine configuration type is **1 vCPU, 3.75 GB RAM** or above. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

5. Run `jupyter notebook` in a shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.

## **Set up your GCP project**

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Enable AutoML API.](https://console.cloud.google.com/apis/library/automl.googleapis.com?q=automl)


## **PIP Install Packages and dependencies**

Install addional dependencies not installed in Notebook environment

In [0]:
! pip install --quiet google-cloud-automl # Use the latest major GA version of the framework.
! pip install matplotlib

**Note:** Try installing using `sudo`, if the above command throw any permission errors.

`Restart` the kernel to allow automl_v1beta1 to be imported for Jupyter Notebooks.


In [0]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## **Set up your GCP Project Id**

Enter your `Project Id` in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

In [0]:
PROJECT_ID = "[your-project-id]" #@param {type:"string"}
COMPUTE_REGION = "us-central1" # Currently only supported region.

## **Authenticate your GCP account**

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

Otherwise, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **AutoML > AutoML Admin**,
   **Storage > Storage Object Admin** and **BigQuery > BigQuery Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [0]:
# Upload the downloaded JSON file that contains your key.
import sys

if 'google.colab' in sys.modules:    
  from google.colab import files
  keyfile_upload = files.upload()
  keyfile = list(keyfile_upload.keys())[0]
  %env GOOGLE_APPLICATION_CREDENTIALS $keyfile
  ! gcloud auth activate-service-account --key-file $keyfile

***If you are running the notebook locally***, enter the path to your service account key as the `GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell

In [0]:
# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.

%env GOOGLE_APPLICATION_CREDENTIALS /path/to/service/account
! gcloud auth activate-service-account --key-file '/path/to/service/account'

## **Create a Cloud Storage bucket**

**The following steps are required, regardless of your notebook environment.**

When you submit a training job using the Cloud SDK, you upload a Python package
containing your training code to a Cloud Storage bucket. AI Platform runs
the code from this package. In this tutorial, AI Platform also saves the
trained model that results from your job in the same bucket. You can then
create an AI Platform model version based on this output in order to serve
online predictions.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [0]:
BUCKET_NAME = "[your-bucket-name]" #@param {type:"string"}

**Only if your bucket doesn't exist**: Run the following cell to create your Cloud Storage bucket. Make sure Storage > Storage Admin role is enabled

In [0]:
! gsutil mb -p $PROJECT_ID -l $COMPUTE_REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [0]:
! gsutil ls -al gs://$BUCKET_NAME

## **Import libraries and define constants**

Import relevant packages.


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [0]:
# AutoML library.
from google.cloud import automl_v1beta1 as automl
import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

Populate the following cell with the necessary constants and run it to initialize constants.

In [0]:
#@title Constants { vertical-output: true }

# A name for the AutoML tables Dataset to create.
DATASET_DISPLAY_NAME = 'stockout_data' #@param {type: 'string'}
# The BigQuery Dataset URI to import data from.
BQ_INPUT_URI = 'bq://product-stockout.product_stockout.stockout' #@param {type: 'string'}
# A name for the AutoML tables model to create.
MODEL_DISPLAY_NAME = 'stockout_model' #@param {type: 'string'}

assert all([
    PROJECT_ID,
    COMPUTE_REGION,
    DATASET_DISPLAY_NAME,
    BQ_INPUT_URI,
    MODEL_DISPLAY_NAME,
])

Initialize the client for AutoML and AutoML Tables.

In [0]:
# Initialize the clients.
automl_client = automl.AutoMlClient()
tables_client = automl.TablesClient(project=PROJECT_ID, region=COMPUTE_REGION)

## **Test the set up**

To test whether your project set up and authentication steps were successful, run the following cell to list your datasets in this project.

If no dataset has previously imported into AutoML Tables, you shall expect an empty return.

In [0]:
# List the datasets.
list_datasets = tables_client.list_datasets()
datasets = { dataset.display_name: dataset.name for dataset in list_datasets }
datasets

You can also print the list of your models by running the following cell.

If no model has previously trained using AutoML Tables, you shall expect an empty return.


In [0]:
# List the models.
list_models = tables_client.list_models()
models = { model.display_name: model.name for model in list_models }
models

## **Import training data**



#### **Create dataset**

Select a dataset display name and pass your table source information to create a new dataset.

In [0]:
# Create dataset.
dataset = tables_client.create_dataset(DATASET_DISPLAY_NAME)
dataset_name = dataset.name
dataset

#### **Import data**



You can import your data to AutoML Tables from GCS or BigQuery. For this solution, you will import data from a BigQuery Table. The URI for your table is in the format of `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

The BigQuery Table used for demonstration purpose can be accessed as `bq://product-stockout.product_stockout.stockout`.

See the table schema and dataset description from the README.

In [0]:
# Import data.
import_data_response = tables_client.import_data(
    dataset=dataset,
    bigquery_input_uri=BQ_INPUT_URI,
)
print('Dataset import operation: {}'.format(import_data_response.operation))

# Synchronous check of operation status. Wait until import is done.
print('Dataset import response: {}'.format(import_data_response.result()))

# Verify the status by checking the example_count field.
dataset = tables_client.get_dataset(dataset_name=dataset_name)
dataset

Importing this stockout datasets takes about 10 minutes.

If you re-visit this Notebook, uncomment the following cell and run the command to retrieve your dataset. Replace `YOUR_DATASET_NAME` with its actual value obtained in the preceding cells.

`YOUR_DATASET_NAME` is a string in the format of `'projects/<project_id>/locations/<location>/datasets/<dataset_id>'`.

In [0]:
# dataset_name = '<YOUR_DATASET_NAME>' #@param {type: 'string'}
# dataset = tables_client.get_dataset(dataset_name=dataset_name)

## **Review the specs**
Run the following command to see table specs such as row count.

In [0]:
# List table specs.
list_table_specs_response = tables_client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs.
list_column_specs_response = tables_client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type.
features = [(key, data_types.TypeCode.Name(value.data_type.type_code))
            for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

In [0]:
# Table schema pie chart.
type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1
    
plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()

In the pie chart above, you see this dataset contains three variable types: `FLOAT64` (treated as `Numeric`), `CATEGORY` (treated as `Categorical`) and `STRING` (treated as `Text`). 

## **Update dataset: assign a label column and enable nullable columns**



#### **Get column specs**

AutoML Tables automatically detects your data column type.

There are a total of 120 columns in this stockout dataset.

Run the following command to check the column data type that automaticallyed detected. If columns contains only numerical values, but they represent categories, change that column data type to caregorical by updating your schema.

In addition, AutoML Tables detects `Stockout` to be categorical that chooses to run a classification model.

In [0]:
# Print column data types.
for column in column_specs:
    print(column, '-', column_specs[column].data_type)

#### **Update columns: make categorical**

From the column data type, you noticed `Item_Number`, `Category`, `Vendor_Number`, `Store_Number`, `Zip_Code` and `County_Number` have been autodetected as `FLOAT64` (Numerical) instead of `CATEGORY` (Categorical). 

In this solution, the columns `Item_Number`, `Category`, `Vendor_Number` and `Store_Number` are not nullable, but `Zip_Code` and `County_Number` can take null values.

To change the data type, you can update the schema by updating the column spec.

In [0]:
type_code='CATEGORY' #@param {type:'string'}

# Update dataset.
categorical_column_names = ['Item_Number', 'Category', 'Vendor_Number', 
                            'Store_Number', 'Zip_Code', 'County_Number']

is_nullable = [False, False, False, False, True, True]                      

for i in range(len(categorical_column_names)):
    column_name = categorical_column_names[i]
    nullable = is_nullable[i]
    tables_client.update_column_spec(
        dataset=dataset,
        column_spec_display_name=column_name,
        type_code=type_code,
        nullable=nullable,
    )

#### **Update dataset: Assign a label**
Select the target column and update the dataset.

In [0]:
#@title Update dataset { vertical-output: true }

target_column_name = 'Stockout' #@param {type: 'string'}
update_dataset_response = tables_client.set_target_column(
    dataset=dataset,
    column_spec_display_name=target_column_name,
)
update_dataset_response

## **Creating a model**



#### **Train a model**

Training the model may take one hour or more. To obtain the results with less training time or budget, you can set [`train_budget_milli_node_hours`](https://cloud.google.com/automl-tables/docs/reference/rest/v1beta1/projects.locations.models), which is the train budget of creating this model, expressed in milli node hours i.e. 1,000 value in this field means 1 node hour.

For demonstration purpose, the following command sets the budget as 1 node hour `('train_budget_milli_node_hours': 1000)`. You can increase that number up to a maximum of 72 hours `('train_budget_milli_node_hours': 72000)` for the best model performance.

Even with a budget of 1 node hour (the minimum possible budget), training a model can take more than the specified node hours

You can also select the objective to optimize your model training by setting `optimization_objective`. This solution optimizes the model by maximizing the Area Under the Precision-Recall (PR) Curve.

In [0]:
# The number of hours to train the model.
model_train_hours = 1 #@param {type:'integer'}
# Set optimization objective to train a model.
model_optimization_objective = 'MAXIMIZE_AU_PRC' #@param {type:'string'}

create_model_response = tables_client.create_model(
    MODEL_DISPLAY_NAME,
    dataset=dataset,
    train_budget_milli_node_hours=model_train_hours*1000,
    optimization_objective=model_optimization_objective,
)
operation_id = create_model_response.operation.name

print('Create model operation: {}'.format(create_model_response.operation))

In [0]:
# Wait until model training is done.
model = create_model_response.result()
model_name = model.name
model

If your Colab times out, use `tables_client.list_models()` to check whether your model has been created.

Then uncomment the following cell and run the command to retrieve your model. Replace `YOUR_MODEL_NAME` with its actual value obtained in the preceding cell.

`YOUR_MODEL_NAME` is a string in the format of `'projects/<project_id>/locations/<location>/models/<model_id>'`

In [0]:
#model_name = '<YOUR_MODEL_NAME>' #@param {type: 'string'}
# model = tables_client.get_model(model_name=model_name)

## **Batch prediction**



#### **Initialize prediction**

Your data source for batch prediction can be GCS or BigQuery. For this solution, you will use a BigQuery Table as the input source. The URI for your table is in the format of `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

To write out the predictions, you need to specify a GCS bucket `gs://BUCKET_NAME`.

The AutoML Tables logs the errors in the `errors.csv` file.

**NOTE:** The batch prediction output file(s) will be updated to the GCS bucket that you set in the preceding cells.

In [0]:
#@title Start batch prediction { vertical-output: true, output-height: 200 }
batch_predict_bq_input_uri = 'bq://product-stockout.product_stockout.batch_prediction_inputs' #@param {type:'string'}
batch_predict_gcs_output_uri_prefix = 'gs://{}'.format(BUCKET_NAME)  #@param {type:'string'}

batch_predict_response = tables_client.batch_predict(
    model_name=model_name, 
    bigquery_input_uri=batch_predict_bq_input_uri,
    gcs_output_uri_prefix=batch_predict_gcs_output_uri_prefix,
)
print('Batch prediction operation: {}'.format(batch_predict_response.operation))

# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata

In [0]:
# Check prediction results.
gcs_output_directory = batch_predict_response.metadata.batch_predict_details\
                       .output_info.gcs_output_directory
result_file = gcs_output_directory + 'tables_1.csv'
print('Batch prediction results are stored as: {}'.format(result_file))

## **Cleaning up**

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

In [0]:
# Delete model resource.
tables_client.delete_model(model_name=model_name)

# Delete dataset resource.
tables_client.delete_dataset(dataset_name=dataset_name)

# Delete Cloud Storage objects that were created.
! gsutil -m rm -r gs://$BUCKET_NAME

# If training model is still running, cancel it.
automl_client.transport._operations_client.cancel_operation(operation_id)